In [2]:
## This file implements neural networks and logistic regression on p002ypresabs_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination of over- and under-sampling
## method.
## To deal with overfitting problem, we include both dropout and regularizer when set up layers in neural networks.
## For fully-connected neural networks, the accuracy is 100% for combination data, and 99.32% for over-sampling data.
## For logistic regression, the accuracy is 100% for combination data, and 100% for over-sampling data.
## Since the accuracy scores are pretty high in logistic regression, we further construct random forest models, which 
## are relatively less likely to bring overfitting compared to decision tree.
## For random forest, the accuracy is 100% for combination data, and 100% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 99.4% for combination data, and 98.83% for over-sampling.

In [3]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [4]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p002ypresabs_quant.csv')
df.shape

(255, 1759)

In [5]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [6]:
df['pheno']

0      0.194875
1      0.265250
2      0.440625
3      0.175500
4      0.173625
5      0.270375
6      0.171000
7      0.182500
8      0.278875
9      0.174375
10     0.176250
11     0.411125
12     0.178250
13     0.191500
14     0.164875
15     0.181875
16     0.394125
17     0.364875
18     0.409875
19     0.191875
20     0.236125
21     0.277000
22     0.169875
23     0.171625
24     0.523625
25     0.169500
26     0.381000
27     0.170000
28     0.160750
29     0.389750
         ...   
225    0.230500
226    0.279800
227    0.319500
228    0.424100
229    0.460500
230    0.304750
231    0.193875
232    0.243000
233    0.195750
234    0.184750
235    0.329125
236    0.160000
237    0.157500
238    0.176375
239    0.326750
240    0.355750
241    0.281375
242    0.181875
243    0.174125
244    0.359250
245    0.161375
246    0.185875
247    0.172750
248    0.167750
249    0.179875
250    0.365375
251    0.223500
252    0.170750
253    0.261125
254    0.231000
Name: pheno, Length: 255

In [7]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [8]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [9]:
df['pheno']
df['pheno'].value_counts()

0    244
1     11
Name: pheno, dtype: int64

In [10]:
df.shape

(255, 1759)

In [11]:
df_clean = df.drop(columns=['id'])

In [12]:
df_clean.shape

(255, 1758)

In [13]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 1757) (255,)


In [14]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

Using TensorFlow backend.


[(0, 235), (1, 241)]


In [15]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 244), (1, 244)]


In [16]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [17]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [18]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [19]:
############# Fully-Connected Neural Network ################

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [21]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [22]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [23]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 333 samples, validate on 143 samples
Epoch 1/100
333/333 [==============================] - 0s 539us/step - loss: 2.2505 - accuracy: 0.7087 - val_loss: 0.5751 - val_accuracy: 0.8462
Epoch 2/100
333/333 [==============================] - 0s 97us/step - loss: 2.1204 - accuracy: 0.8408 - val_loss: 0.5304 - val_accuracy: 0.8741
Epoch 3/100
333/333 [==============================] - 0s 105us/step - loss: 1.9491 - accuracy: 0.8649 - val_loss: 0.5339 - val_accuracy: 0.8531
Epoch 4/100
333/333 [==============================] - 0s 97us/step - loss: 2.1956 - accuracy: 0.8589 - val_loss: 0.4814 - val_accuracy: 0.9301
Epoch 5/100
333/333 [==============================] - 0s 97us/step - loss: 1.6437 - accuracy: 0.8949 - val_loss: 0.4635 - val_accuracy: 0.8951
Epoch 6/100
333/333 [==============================] - 0s 93us/step - loss: 1.9998 - accuracy: 0.8859 - val_loss: 0.4603 - val_accuracy: 0.8951
Epoch 7/100
333/333 [==============================] - 0s 95us/step - loss: 2.2141 - acc

333/333 [==============================] - 0s 97us/step - loss: 2.0515 - accuracy: 0.8769 - val_loss: 0.2561 - val_accuracy: 1.0000
Epoch 58/100
333/333 [==============================] - 0s 93us/step - loss: 1.7686 - accuracy: 0.8949 - val_loss: 0.2512 - val_accuracy: 1.0000
Epoch 59/100
333/333 [==============================] - 0s 93us/step - loss: 1.8179 - accuracy: 0.8919 - val_loss: 0.2522 - val_accuracy: 1.0000
Epoch 60/100
333/333 [==============================] - 0s 94us/step - loss: 1.6303 - accuracy: 0.9039 - val_loss: 0.2510 - val_accuracy: 1.0000
Epoch 61/100
333/333 [==============================] - 0s 95us/step - loss: 1.4458 - accuracy: 0.9159 - val_loss: 0.2492 - val_accuracy: 1.0000
Epoch 62/100
333/333 [==============================] - 0s 97us/step - loss: 1.5349 - accuracy: 0.9099 - val_loss: 0.2513 - val_accuracy: 1.0000
Epoch 63/100
333/333 [==============================] - 0s 93us/step - loss: 1.8594 - accuracy: 0.8889 - val_loss: 0.2454 - val_accuracy: 1.000

In [24]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

143/143 [==============================] - 0s 116us/step
combination test accuracy: 100.00%


In [25]:
###### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [26]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 341 samples, validate on 147 samples
Epoch 1/100
341/341 [==============================] - 0s 654us/step - loss: 1.6251 - accuracy: 0.6745 - val_loss: 0.6403 - val_accuracy: 0.7551
Epoch 2/100
341/341 [==============================] - 0s 89us/step - loss: 2.0915 - accuracy: 0.7507 - val_loss: 0.5943 - val_accuracy: 0.7823
Epoch 3/100
341/341 [==============================] - 0s 97us/step - loss: 1.8977 - accuracy: 0.7771 - val_loss: 0.5682 - val_accuracy: 0.7823
Epoch 4/100
341/341 [==============================] - 0s 92us/step - loss: 2.2811 - accuracy: 0.7683 - val_loss: 0.5476 - val_accuracy: 0.8299
Epoch 5/100
341/341 [==============================] - 0s 138us/step - loss: 2.3734 - accuracy: 0.7713 - val_loss: 0.5351 - val_accuracy: 0.8912
Epoch 6/100
341/341 [==============================] - 0s 101us/step - loss: 1.7037 - accuracy: 0.8270 - val_loss: 0.5160 - val_accuracy: 0.8844
Epoch 7/100
341/341 [==============================] - 0s 95us/step - loss: 2.0775 - ac

Epoch 57/100
341/341 [==============================] - 0s 89us/step - loss: 1.6792 - accuracy: 0.8915 - val_loss: 0.2925 - val_accuracy: 0.9932
Epoch 58/100
341/341 [==============================] - 0s 86us/step - loss: 1.9401 - accuracy: 0.8768 - val_loss: 0.3075 - val_accuracy: 0.9864
Epoch 59/100
341/341 [==============================] - 0s 83us/step - loss: 1.9547 - accuracy: 0.8710 - val_loss: 0.3064 - val_accuracy: 0.9932
Epoch 60/100
341/341 [==============================] - 0s 84us/step - loss: 1.8559 - accuracy: 0.8768 - val_loss: 0.2975 - val_accuracy: 0.9932
Epoch 61/100
341/341 [==============================] - 0s 84us/step - loss: 1.7951 - accuracy: 0.8886 - val_loss: 0.2839 - val_accuracy: 0.9932
Epoch 62/100
341/341 [==============================] - 0s 88us/step - loss: 1.9673 - accuracy: 0.8768 - val_loss: 0.2812 - val_accuracy: 0.9932
Epoch 63/100
341/341 [==============================] - 0s 89us/step - loss: 1.5359 - accuracy: 0.9003 - val_loss: 0.2890 - val_ac

In [28]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

147/147 [==============================] - 0s 72us/step
over-sampling test accuracy: 99.32%


In [29]:
############ Logistic Regression ############

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [45]:
###### logistics on combination data
log_comb = LogisticRegression(random_state=123)
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 100.00%


In [43]:
##### logistics on over-sampling data
log_over = LogisticRegression(random_state=123)
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [44]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 100.00%


In [35]:
############## Random Forest ##############

In [36]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100, random_state=123)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [37]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 100.00%


In [38]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [39]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 100.00%


In [40]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [41]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.98507463 0.98507463 1.         1.         1.        ]
0.9940298507462686


In [42]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.98550725 0.98529412 0.98529412 0.98529412 1.        ]
0.9882779198635976
